In [9]:
import csv
import json
import re
import collections
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pprint
pp=pprint.PrettyPrinter(indent=4)

In [10]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
%cd gdrive/My Drive/추천

/content/gdrive/My Drive/추천
[Errno 2] No such file or directory: 'gdrive/My Drive/추천'
/content/gdrive/My Drive/추천


In [12]:
#ls

In [13]:
csvfile = open('query_results2.txt', 'r')
fieldnames = ("product_NAME","tag_NAME")
reader = csv.DictReader(csvfile, fieldnames)
print(reader)

읽은 파일을 저장하기

In [14]:
product_tag_list = collections.defaultdict(list)

for row in reader:
#     print(row['product_NAME'])
    #print(row['product_NAME'], '\n', row['tag_NAME'])
    col_name=['name','keywords']

    product_tag_list[row['product_NAME']].append(row['tag_NAME'])

In [15]:
len(product_tag_list)
type(product_tag_list)

collections.defaultdict

In [16]:
type(product_tag_list)

collections.defaultdict

In [17]:
product_tag_list

defaultdict(list,
            {'1등급 구운란 10구': ['에',
              '하',
              '1',
              '1',
              '1',
              '팩',
              '입',
              '만',
              '어',
              '10',
              '10',
              '을',
              '굽',
              '(',
              '란',
              '선별',
              '구운',
              '등급',
              '등급',
              '구',
              '계란',
              '가마'],
             '6년근 홍삼이 들어간 삼계탕 재료 140g': ['g',
              'g',
              '하',
              '이',
              '1',
              '봉',
              '지',
              '으로',
              '어',
              '재료',
              '재료',
              '건강',
              '탕',
              '탕',
              '년',
              '년',
              '들어가',
              '6',
              '6',
              '근',
              '근',
              '홍삼',
              '홍삼',
              '삼계',
              '삼계',
              '140',
       

In [18]:
dic = dict(product_tag_list)

In [19]:
for d in dic:
  print(d)

1등급 구운란 10구
6년근 홍삼이 들어간 삼계탕 재료 140g
GAP 가지 2입
GAP 네불라 토마토 500g
GAP 다다기 오이 7입
GAP 대봉 반건시 4입
GAP 대봉감말랭이 300g
GAP 대저 짭짤이 토마토
GAP 대저 짭짤이 토마토 2.5kg
GAP 대저 토마토 2.5kg
GAP 대저 토마토 750g
GAP 대추 방울토마토 750g
GAP 대추방울토마토 1.2kg
GAP 대추방울토마토 1kg
GAP 델라웨어 포도 200g
GAP 미니오이 10입
GAP 미시마사과 2kg 박스
GAP 밀양 깻잎 3속
GAP 방울토마토 500g
GAP 부사 사과 3입
GAP 산딸기 250g
GAP 새싹삼 12뿌리
GAP 성주참외 2kg
GAP 세지 멜론 1.8kg
GAP 스위텔 토마토 500g
GAP 신고배 2입
GAP 신틸라 블루베리 100g
GAP 실속 꼬마 참외 5~6입
GAP 씻어나온 사과 2종
GAP 애플 수박
GAP 애호박 1개
GAP 연근 1통
GAP 오이 2입
GAP 유레카 블루베리 100g
GAP 인증 사과 한봉지 5~6입
GAP 적상추 150g
GAP 철원 오대미 3종
GAP 청상추 100g
GAP 추부 깻잎 30g
GAP 칵테일 토마토 800g
GAP 토마토 2kg
GAP 파프리카 3종
GAP 하우스 감귤 2종
GAP 황금 토마토 500g
GAP 흑토마토 800g
Hallo! 노르웨이 고등어 (3개입)
Tin case 건견과 10종
[1855] 미국산 블랙 앵거스 안심 스테이크 200g(냉장)
[1855] 미국산 블랙 앵거스 안심 찹스테이크 200g(냉장)
[1855] 미국산 블랙앵거스 꽃갈비살 구이 200g(냉장)
[1855] 미국산 블랙앵거스 부채살 구이 200g(냉장)
[1855] 미국산 블랙앵거스 부채살 스테이크 200g(냉장)
[1855] 미국산 블랙앵거스 살치살 구이 200g(냉장)
[8도포차] 대구식 연탄 돼지불고기
[8도포차] 대파 품은 초벌막창
[8도포차] 매운 닭오돌살 직화구이
[8도포차] 수제 매운 오돌갈비
[8도포차] 수제 양념

Data Cleaning: 태그, 상품 설명 정제하기

In [37]:
text_to_remove=['/','.','&','!','(',')','g','kg','~','%','1','2','의','가','안','으로','습니다','는','를','ㅁ','이','없이','어'] #제거할 태그(특수문자, 단위, 조사 등)

dic2={}

for key,val in dic.items():
  for k in key.split():
    val.append(k)
  val=list(set(val)) #중복 제거
  for i in range(len(text_to_remove)):
    tmp=text_to_remove[i]
    if tmp in val:
      val.remove(tmp)
  for n in val:
    s=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', n)
    t=re.sub('\d+g','',s)
    #print(n,t)
    val.remove(n)
    val.append(t)
  for n in val:
    if n.isdigit():
      val.remove(n)
      #print(n)
      continue
      #print(n)
    if 'g' in n:
      val.remove(n)
      continue
  if '' in val:
    val.remove('')
  val.sort()
  #print(key,val)
  dic2[key]=val
  #print(val)

In [38]:
 doc_list=[]
 for key, val in dic2.items():
    doc_list_row = []
    col_name = ['name','keywords']
    
    doc_list_row.append(key)
    doc_list_row.append(val)
    
    doc_list.append(doc_list_row)
    print("key = {key}, value={value}".format(key=key,value=val))

key = 1등급 구운란 10구, value=['10구', '1등급', '가마', '계란', '구', '구운', '구운란', '굽', '등급', '란', '만', '선별', '에', '을', '입', '팩', '하']
key = 6년근 홍삼이 들어간 삼계탕 재료 140g, value=['6년근', '건강', '근', '년', '더욱', '들어가', '들어간', '봉', '삼계', '삼계탕', '재료', '지', '탕', '하', '홍삼', '홍삼이']
key = GAP 가지 2입, value=['2입', 'GAP', '가지', '고', '낮', '봉', '수분', '입', '칼로리', '풍부하']
key = GAP 네불라 토마토 500g, value=['GAP', '고', '네불라', '단단하', '새콤달콤하', '토마토']
key = GAP 다다기 오이 7입, value=['7입', 'GAP', '과', '기', '다다기', '빛', '생', '식감', '오이', '입', '청량', '초록', '하']
key = GAP 대봉 반건시 4입, value=['4입', 'GAP', '감', '곶', '과', '대봉', '매력', '반건', '반건시', '번', '시', '에', '을', '입', '한', '홍시']
key = GAP 대봉감말랭이 300g, value=['GAP', '간식', '감말랭이', '고', '달큰하', '대봉', '대봉감말랭이', '빛', '주홍', '쫀득하']
key = GAP 대저 짭짤이 토마토, value=['GAP', '대저', '제철', '지금', '짭잘달콤새콤', '짭짤', '짭짤이', '토마토', '팩']
key = GAP 대저 짭짤이 토마토 2.5kg, value=['GAP', '게', '넉넉하', '달콤', '대저', '맛', '박스', '새콤', '을', '짭짤', '짭짤이', '짭짤하', '토마토']
key = GAP 대저 토마토 2.5kg, value=['GAP', '기', '달콤짭짤하', '대', '대저', '로', '

In [39]:
doc_list

[['1등급 구운란 10구',
  ['10구',
   '1등급',
   '가마',
   '계란',
   '구',
   '구운',
   '구운란',
   '굽',
   '등급',
   '란',
   '만',
   '선별',
   '에',
   '을',
   '입',
   '팩',
   '하']],
 ['6년근 홍삼이 들어간 삼계탕 재료 140g',
  ['6년근',
   '건강',
   '근',
   '년',
   '더욱',
   '들어가',
   '들어간',
   '봉',
   '삼계',
   '삼계탕',
   '재료',
   '지',
   '탕',
   '하',
   '홍삼',
   '홍삼이']],
 ['GAP 가지 2입', ['2입', 'GAP', '가지', '고', '낮', '봉', '수분', '입', '칼로리', '풍부하']],
 ['GAP 네불라 토마토 500g', ['GAP', '고', '네불라', '단단하', '새콤달콤하', '토마토']],
 ['GAP 다다기 오이 7입',
  ['7입', 'GAP', '과', '기', '다다기', '빛', '생', '식감', '오이', '입', '청량', '초록', '하']],
 ['GAP 대봉 반건시 4입',
  ['4입',
   'GAP',
   '감',
   '곶',
   '과',
   '대봉',
   '매력',
   '반건',
   '반건시',
   '번',
   '시',
   '에',
   '을',
   '입',
   '한',
   '홍시']],
 ['GAP 대봉감말랭이 300g',
  ['GAP', '간식', '감말랭이', '고', '달큰하', '대봉', '대봉감말랭이', '빛', '주홍', '쫀득하']],
 ['GAP 대저 짭짤이 토마토',
  ['GAP', '대저', '제철', '지금', '짭잘달콤새콤', '짭짤', '짭짤이', '토마토', '팩']],
 ['GAP 대저 짭짤이 토마토 2.5kg',
  ['GAP',
   '게',
   '넉넉하',
   '달콤',
   '대저',
   '맛',
  

In [23]:
#df = pd.DataFrame.from_dict(dic2, orient='index')

데이터프레임 형태로 변경

In [24]:
keyword_df = pd.DataFrame(doc_list, columns=col_name)
keyword_df

,name,keywords
0,1등급 구운란 10구,"[10구, 1등급, 가마, 계란, 구, 구운, 구운란, 굽, 등급, 란, 만, 선별..."
1,6년근 홍삼이 들어간 삼계탕 재료 140g,"[6년근, 건강, 근, 년, 더욱, 들어가, 들어간, 봉, 삼계, 삼계탕, 재료, ..."
2,GAP 가지 2입,"[2입, GAP, 가지, 고, 낮, 봉, 수분, 입, 칼로리, 풍부하]"
3,GAP 네불라 토마토 500g,"[GAP, 고, 네불라, 단단하, 새콤달콤하, 토마토]"
4,GAP 다다기 오이 7입,"[7입, GAP, 과, 기, 다다기, 빛, 생, 식감, 오이, 입, 청량, 초록, 하]"
...,...,...
2560,홍고추 80g,"[고추, 궁금하, 맛, 빨갛, 요리, 위하, 홍, 홍고추]"
2561,황태포 5미,"[5미, 과, 만들, 미, 바다, 바람, 볕, 선물, 햇, 황태포]"
2562,황토방 숙성 건조 감말랭이 55g,"[간식, 감말랭이, 건조, 방, 봉, 손이가, 숙성, 영양, 자꾸만, 쪽득쫀득, 황..."
2563,흙생강 100g,"[100g, 손질, 신선하, 않, 알싸하, 지, 하, 흙생강]"


keywords 리스트 원소들을 공백으로 이어붙임

In [25]:
keyword_df['keywords'] = keyword_df['keywords'].apply(lambda x: (' ').join(x))
keyword_df

,name,keywords
0,1등급 구운란 10구,10구 1등급 가마 계란 구 구운 구운란 굽 등급 란 만 선별 에 을 입 팩 하
1,6년근 홍삼이 들어간 삼계탕 재료 140g,6년근 건강 근 년 더욱 들어가 들어간 봉 삼계 삼계탕 재료 지 탕 하 홍삼 홍삼이
2,GAP 가지 2입,2입 GAP 가지 고 낮 봉 수분 입 칼로리 풍부하
3,GAP 네불라 토마토 500g,GAP 고 네불라 단단하 새콤달콤하 토마토
4,GAP 다다기 오이 7입,7입 GAP 과 기 다다기 빛 생 식감 오이 입 청량 초록 하
...,...,...
2560,홍고추 80g,고추 궁금하 맛 빨갛 요리 위하 홍 홍고추
2561,황태포 5미,5미 과 만들 미 바다 바람 볕 선물 햇 황태포
2562,황토방 숙성 건조 감말랭이 55g,간식 감말랭이 건조 방 봉 손이가 숙성 영양 자꾸만 쪽득쫀득 황토 황토방
2563,흙생강 100g,100g 손질 신선하 않 알싸하 지 하 흙생강


In [26]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
keyword_mat = count_vect.fit_transform(keyword_df['keywords']) # 피처 벡터화된 행렬 추출

In [27]:
keyword_mat.shape

(2565, 21028)

피처 벡터화된 행렬에 코사인 유사도를 적용

In [28]:
keyword_sim = cosine_similarity(keyword_mat, keyword_mat)
keyword_sim[0,:]

array([1., 0., 0., ..., 0., 0., 0.])

In [29]:
for i in range(10):
  print(keyword_sim[i,:])

[1. 0. 0. ... 0. 0. 0.]
[0. 1. 0. ... 0. 0. 0.]
[0. 0. 1. ... 0. 0. 0.]
[0.         0.         0.10050378 ... 0.         0.         0.        ]
[0.        0.        0.0836242 ... 0.        0.        0.       ]
[0.        0.        0.0836242 ... 0.        0.        0.       ]
[0.         0.         0.07784989 ... 0.17770466 0.         0.        ]
[0.         0.         0.07784989 ... 0.         0.         0.        ]
[0.         0.         0.06917145 ... 0.         0.         0.        ]
[0.         0.         0.10050378 ... 0.         0.         0.        ]


In [30]:
keyword_sim.shape

(2565, 2565)

유사도가 내림차 순으로 정렬된 행렬의 인덱스를 가져오기

In [31]:
keyword_sim_sorted_index = keyword_sim.argsort()[:,::-1]
keyword_sim_sorted_index[:,0] # 모든 행(모든 아이템)에 대해 유사도가 가장 놓은 아이템(첫 열) 인덱스를 추출한다

array([   0,    1,    2, ..., 2562, 2563, 2564])

유사도가 높은 5순위 상품들의 인덱스

In [33]:
index_list = keyword_sim_sorted_index[:,1].tolist() # 0은 자기자신, 1은 그 다음 유사도 높은 아이템 인덱스
index_list2 = keyword_sim_sorted_index[:,2].tolist() 
index_list3 = keyword_sim_sorted_index[:,3].tolist() 
index_list4 = keyword_sim_sorted_index[:,4].tolist() 
index_list5 = keyword_sim_sorted_index[:,5].tolist() 

type(index_list)

list

In [34]:
#pp.pprint(index_list2)

In [35]:
len(index_list)

2565


최초 DataFrame 은 keyword_df.

keyword_df.iloc[숫자]

In [36]:

for i in range(len(index_list)): # 범위 [0,len(index_list))
    # keyword_df.iloc[i]['name'] # 판다스 접근? .? [컬럼명?] : 컬럼명으로 접근 잘된다
    if keyword_df.iloc[i][0]==keyword_df.iloc[index_list[i]][0]:
      continue
    print(">>기준 상품:",keyword_df.iloc[i][0])
    print("키워드:",keyword_df.iloc[i][1]) 
    
    
    print(">유사 상품1:", keyword_df.iloc[index_list[i]][0]) 
    print("키워드:", keyword_df.iloc[index_list[i]][1]) 

    print(">유사 상품2:", keyword_df.iloc[index_list2[i]][0]) 
    print("키워드:", keyword_df.iloc[index_list2[i]][1]) 

    print(">유사 상품3:", keyword_df.iloc[index_list3[i]][0]) 
    print("키워드:", keyword_df.iloc[index_list3[i]][1]) 

    print(">유사 상품4:", keyword_df.iloc[index_list4[i]][0]) 
    print("키워드:", keyword_df.iloc[index_list4[i]][1]) 

    print(">유사 상품5:", keyword_df.iloc[index_list5[i]][0]) 
    print("키워드:", keyword_df.iloc[index_list5[i]][1]) 
    
    print("===================================================================")
    #keyword_df.iloc[index_list[i]]['name']

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
>유사 상품3: 무농약 고사리&도라지 300g
키워드: 고사리 고사리&도라지 농약 무 무농약
>유사 상품4: 무농약 표고칩 40g
키워드: 건 게 내 농약 무 무농약 버섯 썰 앙증맞 칩 표고 표고칩
>유사 상품5: 무농약 통셀러리 500g
키워드: 농약 무 무농약 식감 아삭아삭 청량 통셀러리 하
>>기준 상품: 무농약 애호박 1개
키워드: 1개 개 굽 나물 농약 로 만능 맛있 먹 무 무농약 무치 애 애호박 어도 채소 호박
>유사 상품1: 부추 200g
키워드: 만능 맛있 먹 무치 봉 부추 부치 어도 전 채소
>유사 상품2: 무농약 숙주나물 300g
키워드: 나물 농약 만능 무 무농약 봉 숙주 숙주나물 아삭아삭 하
>유사 상품3: GAP 애호박 1개
키워드: 1개 GAP 개 고 고소 만능 부드럽 애 애호박 채소 하 호박
>유사 상품4: 무농약 파프리카 1개
키워드: 1개 같 개 과일 농약 더 무 무농약 보다 채소 파프리카
>유사 상품5: [김구원선생] 무농약 콩나물 200g
키워드: 김구원 김구원선생 나물 농약 무 무농약 봉 선생 재배 콩 콩나물 하
>>기준 상품: 무농약 양배추 1/2통
키워드: 1/2통 고 남기 농약 다 먹 무 무농약 반통 배추 수 않 양 양배추 있 지 통
>유사 상품1: 무농약 토마토 1kg
키워드: 농약 무 무농약 재배 친 토마토 하 환경
>유사 상품2: 무농약 아마란스 350g
키워드: 곡물 내리 농약 무 무농약 봉 신 아마란스
>유사 상품3: 무농약 은이버섯 200g
키워드: 고 농약 무 무농약 은이버섯 쫄깃하 촉촉하 팩 흰목이버섯
>유사 상품4: 무농약 감자 600g
키워드: 감자 농약 많 무 무농약 봉 수분 아 포실 포실하
>유사 상품5: 무농약 표고칩 40g
키워드: 건 게 내 농약 무 무농약 버섯 썰 앙증맞 칩 표고 표고칩
>>기준 상품: 무농약 양상추 1통
키워드: 1통 고 냉장 농약 매일 먹 무 무농약 상추 양 양상추 채소 통 필수
>유사 상품1: 무농약 스탠포